## Etapas

1. Criação de features
2. Tratamento de outliers
3. Normalização dos dados
4. Separação em treino/teste

In [ ]:
# @title Importação de bibliotecas

# Dataset
import pandas as pd
import numpy as np
import requests, os
from pathlib import Path

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Tratamento de outliers
from scipy.stats.mstats import winsorize
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# @title Carregamento do dataset

permalink = "https://github.com/mfigueireddo/ciencia-de-dados/blob/ba579573c5b8a9246ca04f7da29bc2c74c8b362c/datasets/pre-pipeline_wildfires.parquet"
raw_url = permalink.replace("/blob/", "/raw/")

dest = Path("/content/raw_wildfires.parquet")

with requests.get(raw_url, stream=True) as r:
    r.raise_for_status()
    with open(dest, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)

# Leitura do arquivo .csv
wildfires = pd.read_parquet(dest, engine="pyarrow")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/QuickAccess/pre-pipeline_wildfires.csv'

In [ ]:
# @title 1. Criação de novas features

# --- CÉLULA ÚNICA: CRIAÇÃO DE TODAS AS 4 FEATURES ---

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# 1. Verificar se o DataFrame 'wildfires' existe
if 'wildfires' not in locals():
    print("ERRO: O DataFrame 'wildfires' não foi encontrado.")
    print("Por favor, execute a Célula 1 (Carregamento do Dataset) primeiro.")
else:
    print("--- Iniciando a criação de todas as 4 features ---")

    # 2. PREPARAÇÃO: Garantir que 'data' é datetime (necessário para todas)
    try:
        wildfires['data'] = pd.to_datetime(wildfires['data'])
        print("Coluna 'data' convertida para datetime.")
    except Exception as e:
        print(f"ERRO crítico ao converter 'data': {e}.")

    # ==========================================================
    # FEATURE 1: ESTAÇÃO DO ANO
    # ==========================================================
    print("\nCriando Feature 1: 'estacao_ano'...")
    try:
        if 'estacao_ano' in wildfires.columns:
            wildfires = wildfires.drop(columns=['estacao_ano'])

        mapa_estacoes = {
            12: 'Inverno', 1: 'Inverno', 2: 'Inverno',
            3: 'Primavera', 4: 'Primavera', 5: 'Primavera',
            6: 'Verão', 7: 'Verão', 8: 'Verão',
            9: 'Outono', 10: 'Outono', 11: 'Outono'
        }
        wildfires['estacao_ano'] = wildfires['data'].dt.month.map(mapa_estacoes)
        print("✅ Feature 'estacao_ano' criada com sucesso.")

    except Exception as e:
        print(f"ERRO ao criar 'estacao_ano': {e}")

    # ==========================================================
    # FEATURE 2: REGIÃO DE INCÊNDIO (CLUSTERIZAÇÃO DBSCAN)
    # ==========================================================
    print("\nCriando Feature 2: 'regiao_incendio'...")
    try:
        cols_to_drop = ['lat_cell', 'lon_cell', 'regiao_incendio']
        for col in cols_to_drop:
            if col in wildfires.columns:
                wildfires = wildfires.drop(columns=[col])

        DECIMALS = 2
        wildfires['lat_cell'] = wildfires['latitude'].round(DECIMALS)
        wildfires['lon_cell'] = wildfires['longitude'].round(DECIMALS)

        EPS_REFINADO = 0.015  # Raio de ~1.6km
        MIN_AMOSTRAS = 5       # Mínimo de 5 células vizinhas
        print(f"Parâmetros do DBSCAN: eps={EPS_REFINADO}, min_samples={MIN_AMOSTRAS}")

        areas_unicas = wildfires[['lat_cell', 'lon_cell']].drop_duplicates().reset_index(drop=True)
        print(f"Encontradas {len(areas_unicas)} células únicas para agrupar.")

        dbscan = DBSCAN(eps=EPS_REFINADO, min_samples=MIN_AMOSTRAS, metric='haversine')
        coordenadas_rad = np.radians(areas_unicas[['lat_cell', 'lon_cell']].to_numpy())
        clusters = dbscan.fit_predict(coordenadas_rad)

        areas_unicas['regiao_incendio'] = clusters
        n_regioes = areas_unicas['regiao_incendio'].nunique() - (1 if -1 in clusters else 0)
        n_isolados = (areas_unicas['regiao_incendio'] == -1).sum()
        print(f"Resultado: {n_regioes} regiões de incêndio identificadas e {n_isolados} células isoladas.")

        wildfires = wildfires.merge(
            areas_unicas[['lat_cell', 'lon_cell', 'regiao_incendio']],
            on=['lat_cell', 'lon_cell'],
            how='left'
        )
        print("✅ Feature 'regiao_incendio' criada e mapeada de volta ao DataFrame.")

    except Exception as e:
        print(f"ERRO ao criar 'regiao_incendio': {e}")

    # ==========================================================
    # FEATURES 3 E 4: TENDÊNCIA (MÉDIAS MÓVEIS)
    # ==========================================================
    print("\nCriando Features 3 & 4: Tendência (Médias Móveis)...")
    try:
        # Ordenar os valores (CRUCIAL para cálculos de média móvel)
        wildfires = wildfires.sort_values(by=['fire_id', 'data'])
        print("DataFrame ordenado por 'fire_id' e 'data' (necessário para médias móveis).")

        # Remover colunas antigas se existirem
        if 'soma_precipitacao_14d' in wildfires.columns:
            wildfires = wildfires.drop(columns=['soma_precipitacao_14d'])
        if 'media_temp_max_7d' in wildfires.columns:
            wildfires = wildfires.drop(columns=['media_temp_max_7d'])

        # Criar as features
        print("Calculando 'soma_precipitacao_14d'...")
        wildfires['soma_precipitacao_14d'] = wildfires.groupby('fire_id')['precipitacao'].transform(
            lambda x: x.rolling(window=14, min_periods=1).sum()
        )
        print("Calculando 'media_temp_max_7d'...")
        wildfires['media_temp_max_7d'] = wildfires.groupby('fire_id')['temperatura_max'].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean()
        )
        print("✅ Features de tendência criadas com sucesso!")

    except KeyError as e:
        print(f"ERRO: Faltando uma coluna necessária: {e}.")
    except Exception as e:
        print(f"Um erro inesperado ocorreu: {e}")

    # ==========================================================
    # VALIDAÇÃO FINAL CONSOLIDADA
    # ==========================================================
    print("\n--- Processo completo de criação de features concluído! ---")
    print("Amostra do DataFrame com todas as novas features:")

    colunas_finais = [
        'data', 'fire_id', 'estacao_ano', 'regiao_incendio',
        'soma_precipitacao_14d', 'media_temp_max_7d'
    ]
    # Apenas para garantir que colunas auxiliares não estejam na validação
    colunas_existentes = [col for col in colunas_finais if col in wildfires.columns]

    display(wildfires[colunas_existentes].head(10))

In [ ]:
# @title 2. Tratamento de outliers

log_cols = [
    "precipitacao",
    "velocidade_vento",
    "evapotranspiracao_potencial",
    "deficit_pressao_vapor",
]
sqrt_cols = [
    "umidade_especifica",
    "umidade_combustivel_morto_100_horas",
    "umidade_combustivel_morto_1000_horas",
    "evapotranspiracao_real",
]
winsor_cols = [
    "umidade_relativa_max",
    "umidade_relativa_min",
    "temperatura_min",
    "temperatura_max",
    "indice_queima",
]

class LogSqrtWinsorizer(BaseEstimator, TransformerMixin):

    def __init__(self, log_cols=None, sqrt_cols=None, winsor_cols=None, winsor_limits=(0.05, 0.05)):
        self.log_cols = log_cols or []
        self.sqrt_cols = sqrt_cols or []
        self.winsor_cols = winsor_cols or []
        self.winsor_limits = winsor_limits

    def fit(self, X, y=None):
        Xdf = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)

        self.log_offset_ = {}
        for c in self.log_cols:
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                m = s.min()
                self.log_offset_[c] = (abs(m) + 1) if pd.notna(m) and m <= 0 else 1.0

        self.sqrt_offset_ = {}
        for c in self.sqrt_cols:
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                m = s.min()
                self.sqrt_offset_[c] = (abs(m) + 0.01) if pd.notna(m) and m < 0 else 0.0

        present_winsor = [c for c in self.winsor_cols if c in Xdf.columns]
        if present_winsor:
            q_low, q_high = self.winsor_limits
            self.low_  = pd.to_numeric(Xdf[present_winsor], errors="coerce").quantile(q_low)
            self.high_ = pd.to_numeric(Xdf[present_winsor], errors="coerce").quantile(1 - q_high)
        else:
            self.low_ = pd.Series(dtype=float)
            self.high_ = pd.Series(dtype=float)

        return self

    def transform(self, X):
        Xdf = X.copy() if isinstance(X, pd.DataFrame) else pd.DataFrame(X).copy()

        # LOG
        for c, off in self.log_offset_.items():
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                Xdf[c] = np.log(s + off)

        # SQRT
        for c, off in self.sqrt_offset_.items():
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                Xdf[c] = np.sqrt(s + off)

        # Winsorização
        for c in self.low_.index:
            if c in Xdf.columns:
                s = pd.to_numeric(Xdf[c], errors="coerce")
                Xdf[c] = s.clip(lower=self.low_[c], upper=self.high_[c])

        return Xdf

**Transformação Logarítmica**
- Aplica log(x) ou log(x+constante) para valores positivos
- Muito eficaz para dados com distribuição assimétrica positiva
- Comprime valores grandes e expande valores pequenos
- Fórmula: X_log = log(X + c), onde c evita log(0)

**Transformação Raiz Quadrada**
- Menos drástica que a transformação logarítmica
- Útil para dados de contagem e variáveis positivamente assimétricas
- Fórmula: X_sqrt = sqrt(X)

**Winsorização (Capping/Clipping)**

A **Winsorização** é uma técnica de tratamento de outliers que **limita valores extremos** sem removê-los completamente. Em vez de excluir outliers, substituímos os valores extremos pelos valores de percentis específicos.

**Como funciona:**
- Define-se limites baseados em percentis (ex: 5º e 95º percentil)
- Valores abaixo do limite inferior são substituídos pelo valor do limite inferior
- Valores acima do limite superior são substituídos pelo valor do limite superior

| Variável                                 | Melhor método         | Justificativa (1 linha)                                                                                   |
| :--------------------------------------- | :-------------------- | :-------------------------------------------------------------------------------------------------------- |
| **precipitacao**                         | **Log(x + 1)**        | Reduziu fortemente a assimetria (7.87 → 2.59) e manteve os limites IQR estáveis — ideal para cauda longa. |
| **umidade_relativa_max**                 | **Winsorizar 5%-5%**  | Cortou outliers (23 → 0) e melhorou levemente a simetria; log/sqrt inflaram valores.                      |
| **umidade_relativa_min**                 | **Winsorizar 5%-5%**  | Assimetria e outliers foram totalmente corrigidos (1155 → 0).                                             |
| **umidade_especifica**                   | **Sqrt**              | Melhor simetria (0.89 → 0.16) e forte redução de outliers (6967 → 2102).                                  |
| **radiacao_solar**                       | **Sem transformação** | Já simétrica e sem outliers; log piorou, winsor apenas repete.                                            |
| **temperatura_min**                      | **Winsorizar 5%-5%**  | Remoção completa de outliers (5066 → 0) e leve ganho de simetria.                                         |
| **temperatura_max**                      | **Winsorizar 5%-5%**  | Mesmo comportamento de `temperatura_min`.                                                                 |
| **velocidade_vento**                     | **Log(x + 1)**        | Reduziu assimetria (1.23 → 0.19) e outliers (8723 → 1128) sem eliminar extremos reais.                    |
| **indice_queima**                        | **Winsorizar 5%-5%**  | Log e sqrt aumentaram outliers via IQR; winsor eliminou-os com mínima distorção.                          |
| **umidade_combustivel_morto_100_horas**  | **Sqrt**              | Forte queda de outliers (44 → 9) e leve suavização de forma.                                              |
| **umidade_combustivel_morto_1000_horas** | **Sqrt**              | Reduziu outliers (373 → 4) mantendo distribuição coerente.                                                |
| **componente_energia_lancada**           | **Sem transformação** | Já equilibrada; transformações criam falsos outliers.                                                     |
| **evapotranspiracao_real**               | **Sqrt**              | Melhorou drasticamente a simetria (0.71 → -0.00) e reduziu outliers (3292 → 153).                         |
| **evapotranspiracao_potencial**          | **Log(x + 1)**        | Skew caiu (0.53 → -0.36) e outliers zeraram (679 → 0).                                                    |
| **deficit_pressao_vapor**                | **Log(x + 1)**        | Alta cauda direita suavizada (1.46 → 0.52) e outliers despencaram (14758 → 672).                          |


In [ ]:
# @title 3. Normalização dos dados

In [ ]:
# @title 4. Separação em treino/teste

def group_time_series_cv(
    df: pd.DataFrame,
    time_col: str,
    group_col: str,
    *,
    n_splits: int = 5,
    test_groups_size: int = 1,
    gap_groups: int = 0,
    expanding: bool = True,
    min_train_groups: int | None = None,
    step_groups: int | None = None,
):
    """
    Gera folds (train_idx, test_idx) respeitando ordem temporal por grupo,
    embargo em nível de grupo e exclusão mútua treino/teste por grupo.
    """
    # 1) Ordena grupos pelo primeiro timestamp
    first_time = (
        df[[group_col, time_col]]
        .dropna(subset=[time_col])
        .groupby(group_col)[time_col]
        .min()
        .sort_values()
    )
    ordered_groups = first_time.index.to_numpy()
    n_groups = len(ordered_groups)

    if step_groups is None:
        step_groups = test_groups_size
    if min_train_groups is None:
        # mínimo sensato: pelo menos o tamanho do primeiro teste
        min_train_groups = max(1, test_groups_size)

    # Âncora: último grupo incluso no treino
    # Precisamos garantir espaço para gap + teste à frente
    max_anchor = n_groups - gap_groups - test_groups_size
    if max_anchor <= min_train_groups:
        return  # não há splits possíveis

    splits = 0
    anchor = min_train_groups
    while anchor <= max_anchor and splits < n_splits:
        if expanding:
            train_groups = ordered_groups[:anchor]
        else:
            start = max(0, anchor - min_train_groups)
            train_groups = ordered_groups[start:anchor]

        test_start = anchor + gap_groups
        test_end = test_start + test_groups_size
        test_groups = ordered_groups[test_start:test_end]

        train_idx = df.index[df[group_col].isin(train_groups)].to_numpy()
        test_idx  = df.index[df[group_col].isin(test_groups)].to_numpy()

        if train_idx.size and test_idx.size:
            yield (train_idx, test_idx)
            splits += 1

        anchor += step_groups


## Time Series Cross Validation

A Time Series Cross Validation é uma técnica especializada para validar modelos quando os dados possuem ordem cronológica. Diferente das técnicas tradicionais, ela respeita a estrutura temporal dos dados.

In [ ]:
# @title Pipeline

preprocess = Pipeline(steps=[
    ("features", FunctionTransformer(build_features,  validate=False)),
    ("outliers", LogSqrtWinsorizer(
        log_cols=log_cols,
        sqrt_cols=sqrt_cols,
        winsor_cols=winsor_cols,
        winsor_limits=(0.05, 0.05),    # mantenha seus limites
    )),
    ("scaler",   FunctionTransformer(scale_features,  validate=False)),
])
